In [1]:
from scipy import integrate
warpSize = 32
# eccentricity Adams and Horton 2003 Fig.4
Ap = 9.5
Bp = 0.95
Pp = -1.03 
Ae = 188 
Be = 6.45
Pe = -1.85
E0 = 0.0
# traveling wave at eccentricity of 1.8 and 3.4 deg, Wilson et al 2001 Nat
max_ecc = 2 #deg 
E1 = max_ecc
vertical_slice = lambda E: 180*Ap*(E+Bp)**Pp*Be*(E+Be)**Pe # mm^2/deg
result = integrate.quad(vertical_slice,E0,E1) # mm^2
area_CMF = ((int(result[0])+warpSize-1)//warpSize) * warpSize

In [80]:
import math
## O'Kusky 1982; Kelly & Hawken 2017; M.Schmidt 2018, assuming constant density near the fovea
surface_den_L4Cbeta = 32768 # per mm^2
n_L4Cbeta = area_CMF*surface_den_L4Cbeta # 32^4 * 10 ~ 10.4 million
surface_den_L23 = 20480 # per mm^2
n_L23 = area_CMF*surface_den_L23
print(n_4Cbeta, n_L23)

10485760 6553600


In [76]:
# evenly drop the number of neurons in the area, consider 3D 

In [2]:
area_CMF

320